In [5]:
import json
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from config import Config

In [6]:
class DataIngestor:
    def __init__(self, qdrant_client):
        self.client = qdrant_client
        print(f"⏳ Đang tải model {Config.EMBEDDING_MODEL}...")
        self.encoder = SentenceTransformer(Config.EMBEDDING_MODEL)
        print("✅ Model đã sẵn sàng!\n")
    
    def load_menu(self, path):
        """Load dữ liệu menu món ăn của nhà hàng"""
        try:
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            print(f"👀 [Giai đoạn 1 - Đọc File] Đã đọc menu.json: Tìm thấy {len(data)} danh mục.")
            
            docs = []
            for category in data:
                cat_name = category['category']
                for item in category['items']:
                    # Tạo nội dung ngữ nghĩa cho vector search
                    content = (
                        f"Món ăn {item['name_vn']} ({item['name_en']}). "
                        f"Thuộc loại {cat_name}. "
                        f"Giá {item['price']} VND. "
                        f"Ghi chú {item['note']}."
                    )
                    print(f"   👉 [Giai đoạn 2 - Biến đổi] JSON -> Text: \"{content}\"")
                    docs.append({"text": content, "id": item['id'], "source": "menu"})
            return docs
        except FileNotFoundError:
            print("Warning: Menu file not found.")
            return []
    
    def load_info(self, path):
        """Load dữ liệu thông tin của nhà hàng"""
        try:
            with open(path, 'r', encoding='utf-8') as f:
                data = f.read()
            # Chia nhỏ thông tin nhà hàng thành các đoạn
            chunks = data.split('\n')
            docs = []
            print(f"👀 [Giai đoạn 1 - Đọc File] Đã đọc info.txt.")
            for i, chunk in enumerate(chunks):
                if chunk.strip():
                    print(f"   👉 [Giai đoạn 2 - Biến đổi] Raw Text -> Chunk: \"{chunk.strip()}\"")
                    docs.append({"text": chunk.strip(), "source": "info", "id": f"info_{i}"})
            return docs
        except FileNotFoundError:
            print("Warning: Info file not found.")
            return []
    
    def ingest(self):
        """Nạp dữ liệu vào Vector DB"""
        print("--- Bắt đầu nạp dữ liệu vào Vector DB ---")
        menu_data = self.load_menu(os.path.join(Config.DATA_DIR, "menu.json"))
        print(os.path.join(Config.DATA_DIR, "menu.json"))
        print(Config.BASE_DIR)
        info_data = self.load_info(os.path.join(Config.DATA_DIR, 'restaurant_info.txt'))
        print(os.path.join(Config.DATA_DIR, "restaurant_info.txt"))
        
        all_docs = menu_data + info_data
        
        if not all_docs: 
            print("Không có dữ liệu để nạp.")
            return
        
        # Tạo collection trong Qdrant
        if self.client.collection_exists(collection_name=Config.COLLECTION_NAME):
            self.client.delete_collection(collection_name=Config.COLLECTION_NAME)
            
        self.client.create_collection(
            collection_name=Config.COLLECTION_NAME,
            vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
        )
        print(f"\n⚡ [Giai đoạn 3 - Vector hóa] Đang biến đổi {len(all_docs)} đoạn văn thành số...")
        
        # Vector hóa và upload
        points = []
        embeddings = self.encoder.encode([d['text'] for d in all_docs])
        print(f"   👉 Vector mẫu (3 số đầu/384): {embeddings[0]} ... {embeddings.shape}")
        # Loop tạo PointStruct (kết hợp ID + Vector + Payload)
        for i, doc in enumerate(all_docs):
            points.append(PointStruct(
                id=i,
                vector=embeddings[i].tolist(),
                payload=doc
            ))
        
        print(f"\n📦 [Giai đoạn 4 - Đóng gói] Chuẩn bị đẩy {len(points)} Points vào Qdrant {points}.")
        # Client.upsert -> Đẩy lên Qdrant
        self.client.upsert(
            collection_name=Config.COLLECTION_NAME,
            points=points
        )
        print(f"--- Đã nạp {len(points)} documents vào Qdrant ---")

In [7]:
if __name__ == "__main__":
    # Dùng Qdrant chạy trên RAM (không cần cài Docker để test)
    Client = QdrantClient(":memory:")
    
    ingestor = DataIngestor(Client)
    ingestor.ingest()
    
    # Test thử tìm kiếm để chứng minh dữ liệu đã vào
    print("\n--- 🔎 TEST TÌM KIẾM ---")
    query = "Tôi muốn ăn món gì đó cay cay"
    print(f"Câu hỏi: {query}")
    
    model = SentenceTransformer(Config.EMBEDDING_MODEL)
    hits = Client.search(
        collection_name=Config.COLLECTION_NAME,
        query_vector=model.encode(query).tolist(),
        limit=2
    )
    
    for hit in hits:
        print(f"Kết quả tìm thấy: {hit.payload['text']}")
    

⏳ Đang tải model AITeamVN/Vietnamese_Embedding...
✅ Model đã sẵn sàng!

--- Bắt đầu nạp dữ liệu vào Vector DB ---
👀 [Giai đoạn 1 - Đọc File] Đã đọc menu.json: Tìm thấy 6 danh mục.
   👉 [Giai đoạn 2 - Biến đổi] JSON -> Text: "Món ăn Gỏi gà Vân Nam (Yunnan Shredded Chicken Salad). Thuộc loại Khai vị (Appetizer). Giá 138000 VND. Ghi chú Cay/Spicy."
   👉 [Giai đoạn 2 - Biến đổi] JSON -> Text: "Món ăn Đậu hũ hoàng kim (Deep Fried Tofu with Salted Egg Yolk). Thuộc loại Khai vị (Appetizer). Giá 88000 VND. Ghi chú Chef's Signature."
   👉 [Giai đoạn 2 - Biến đổi] JSON -> Text: "Món ăn Càng cua bách hoa (Bai Hua Crab Claw). Thuộc loại Khai vị (Appetizer). Giá 205000 VND. Ghi chú Chef's Signature."
   👉 [Giai đoạn 2 - Biến đổi] JSON -> Text: "Món ăn Nem cuốn tứ vị (Spring Rolls With Four Flavors). Thuộc loại Khai vị (Appetizer). Giá 130000 VND. Ghi chú ."
   👉 [Giai đoạn 2 - Biến đổi] JSON -> Text: "Món ăn Bánh bao kim sa (Salted Egg Bun). Thuộc loại Điểm tâm (Dimsum & Buns). Giá 88000 VND. Ghi c